# YOUR PROJECT TITLE

Imports and set magics:

In [ ]:
import numpy as np
from scipy import optimize
import sympy as sm
from sympy import Symbol
from sympy.solvers import solve
sm.init_printing(use_unicode=True) # for pretty printing
from IPython.display import display
import matplotlib.pyplot as plt # baseline modul
import ipywidgets as widgets

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject


# The Solow model with human capital

We will be working with a solowmodel with fysical and human capital. 
Like fysical capital in the standard solow model, human capital accumulates. A fixed rate depriciates each priode, and a constant fraction $s_H$ of production is turned in to human capital. human capital can be thought of as things such as education, health and IQ.  


## Variables 

* $Y_t$  : Production
* $L_t$ : Labor
* $A_t$ : Technology
* $K_t$ : Phisical Capital
* $H_t$ : Human Capital

## Parameters

* $s_h$:       Savings/Investments in human capital\
* $s_k$:       Savings/Investments in physical capital\
* $g$:         Growth rate of technology\
* $n$:         Growth rate of labour force
* $\delta$:     Deprisiation rate of physical capital\
* $\alpha$:     Return physical capital\
* $\phi$:       Return on human capital\
* $h$:         Human capital per effective worker (Stock)\

## Equations 

**Production funciton** 

$ Y_{t}=K_{t}^{\alpha}H_{t}^{\varphi}(A_{t}L_{t})^{1-\alpha-\varphi} $

**Saving** \
Constant fraction of production in the eoconomy

$ S_t = (s_H+s_K)Y_t,\, s_H+s_K\in(0,1) $

**Capital Accumulation**

$ K_{t+1}=s_{K}Y_t+(1-\delta)K_{t}$

**Human capital accumulation** 

$ H_{t+1} = s_{H}Y_t+(1-\delta)H_t $

**Population and teknology (TFP) growth**

$ L_{t+1} = (1+n)L_t $

$ A_{t+1} = (1+g)A_t $

## Tilde equations

In order to analyse the model we calculate the per effectiv worker values. That is, divid by the population and teknology level. 


**Defitiontions**

Production, fysical capital and human capital as divided by popluation and teknology to get .... \
$   \tilde{y}_{t} = \frac{Y_{t}} {A_tL_t} $\
$   \tilde{k}_{t} = \frac{K_{t}} {A_tL_t} $\
$   \tilde{h}_{t} = \frac{H_{t}} {A_tL_t} $

**Transition Equations** 

The tilde transition eqautions can be found by inserting the tilde definitions ...

$ \tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}[s_K\tilde{k}_{t}^{\alpha}\tilde{h}_{t}^{\varphi}+(1-\delta)\tilde{k}_{t}] $

$ \tilde{h}_{t+1}= \frac{1}{(1+n)(1+g)}[s_H\tilde{k}_{t}^{\alpha}\tilde{h}_{t}^{\varphi}+(1-\delta)\tilde{h}_{t}] $



**OBS: In the folowing, the tilde values will simply be writen as uncapitalized letters**



## Analytical solution

## Nulclines

1. Defining the variables as symbols for using sympy

In [ ]:

k = sm.symbols('k')
h = sm.symbols('h')
y = sm.symbols('y')
c = sm.symbols('c')
sh = sm.symbols('s_h')
sk = sm.symbols('s_k')
delta = sm.symbols('delta')
phi = sm.symbols('phi')
alpha = sm.symbols('alpha')
g = sm.symbols('g')
n = sm.symbols('n')
A = sm.symbols('A')

x = [sk, sh, g, phi, n, alpha, delta]


2. Function for deriving the analytical solution

## Numerical solution

We now progress to the numerical solution to the Solow-Swan-model with human capital accumulation


In [ ]:
#For our numerical solution we use the following baseline parameters

s_h = 0.1
s_k = 0.1
g = 0.02
n = 0.01
alpha = 1/3
phi = 1/3
delta = 0.06

In [ ]:
def f(h,k,s_h,s_k,g,n,alpha,phi,delta):
    """input arguments:
    h     (float): Human capital per effective worker (Stock)
    k     (float): Physical capital per effective worker (Stock)
    s_h   (float): Savings/Investments in human capital
    s_k   (float): Savings/Investments in physical capital
    g     (float): Growth rate of technology
    n     (float): Growth rate of labour force
    delta (float): Depreciation rate of physical capital
    alpha (float): Return on physical capital
    phi   (float): Return on human capital
    
    Returns: 
    The transition equation for human and physical capital per effective worker in steady state
    """

    h_function = 1/((1+n)*(1+g))*(s_h*k**(alpha)*h**(phi)+(1-delta)*h)-h
    k_function = 1/((1+n)*(1+g))*(s_k*k**(alpha)*h**(phi)+(1-delta)*k)-k
    return h_function,k_function

# Setting up the objective and solving the model
objective = lambda x: [f(x[0],x[1],s_h,s_k,g,n,alpha,phi,delta)]
sol = optimize.root(objective,[1,1],method = 'broyden1')
num_sol = sol.x

print(f'The level of human and physical capital per effective worker in steady state is = {num_sol[0]:.3f} and {num_sol[1]:.3f}, respectively.')


The phase-diagram can be used to illustrate that there only exist a single point where both human and physical capital per effective worker is constant, which is characterized as our steady state:

$\Delta\tilde{h}^{\ast}=\Delta\tilde{k}^{\ast}=0$ 


The phase-diagram consists of two nullclines where $\Delta\tilde{h}^{\ast}=0$ and $\Delta\tilde{k}^{\ast}=0$, repsectively.

In [ ]:
def solve_steadystate(s_h,s_k,g,n,alpha,phi,delta):
    """input arguments:
    s_h   (float): Savings/Investments in human capital
    s_k   (float): Savings/Investments in physical capital
    g     (float): Growth rate of technology
    n     (float): Growth rate of labour force
    alpha (float): Return on physical capital
    phi   (float): Return on human capital
    delta (float): Depriciation rate of physical capital
    
    Returns:
    The nullclines for physical and human capital, respectively
    """
    
    # a. Setting up the grid
    grid_space = np.linspace(1e-5,2.5,250)
    k_vector = np.empty(250)
    h_vector  = np.empty(250)
    
    # b. solve for each k in grid
    for i,k in enumerate(grid_space):
        #Solve for constant human capital
        
        # Objective and optimizer
        obj = lambda h: -h_function(k,s_h,s_k,g,n,alpha,phi,delta,h)
        result = optimize.root_scalar(obj,method='brentq',bracket=[1e-20,50])
        h_vector[i] = result.root

        #Solve for constant physical capital

        # Objective and optimizer
        obj = lambda h: -k_function(h,s_h,s_k,g,n,alpha,phi,delta,k)
        result = optimize.root_scalar(obj,method='brentq',bracket=[1e-20,50])
        k_vector[i] = result.root

    return grid_space, k_vector, h_vector


#Solve the function
grid_space, k_vector, h_vector = solve_steadystate(s_h,s_k,g,n,alpha,phi,delta)


#Create the plots
fig = plt.figure(figsize=(13,5))
ax = fig.add_subplot(1,2,1)
ax.plot(grid_space,k_vector, label=r'$\Delta \tilde{k}=0$')
ax.plot(grid_space,h_vector, label=r'$\Delta \tilde{h}=0$')


az = fig.add_subplot(1,2,2)
az.plot(grid_space,k_vector, label=r'$\Delta \tilde{k}=0$')
az.plot(grid_space,h_vector, label=r'$\Delta \tilde{h}=0$')


#Marking the steady state points
ax.scatter(sol.x[1],sol.x[0],color='red',s=100,zorder=3, label=r'$\Delta\tilde{h}=\Delta\tilde{k}=0$')

ax.axvline(sol.x[1],ymax=0.55,color='gray',linestyle='--') 
ax.axhline(sol.x[0],xmax=0.55,color='gray',linestyle='--')

#Legends
ax.set_xlabel(r'Human capital per effective worker')
ax.set_ylabel(r'Physical capital per effective worker')
ax.legend() 

#The axis values are chosen
ax.set(xlim=(0, 2.5), ylim=(0, 2.5))
az.set(xlim=(0, 2.5), ylim=(0, 2.5))

#Titles
ax.set_title('Base scenario');
az.set_title('Higher initial level of physical capital');

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 